# prototype

Here we create an `Ident` class: an object that represents a node, a single identity, in the network.

A single idenity can sign, verify, encrypt and decrypt transactions. It can produce, recieve, and broadcast transactions. It can interface with the running memory of the network provided by the permissioned consensus algorithm and analyse the results in order to calculate who is a valid identity and who is not.

## Basic Cryptography Functionality

In [2]:
# object to serve as a prototype node, with all the abilities of a node
class Ident():
    def __init__(self):
        self.private, self.public = Ident.gen_ident()
        everyone = []
    
    ### Encryption #########################################################################################
    
    @staticmethod
    def gen_ident():
        from cryptography.hazmat.backends import default_backend
        from cryptography.hazmat.primitives.asymmetric import rsa
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
            backend=default_backend()
        )
        public_key = private_key.public_key()
        return private_key, public_key
    
    @staticmethod
    def to_bytes(string: str):
        if isinstance(string, str):
            return str.encode(string)
        if isinstance(string, bytes):
            return string
        raise TypeError('string to be converted to bytes is not of str or bytes type')
        
    @staticmethod
    def padding_OAEP():
        from cryptography.hazmat.primitives import hashes
        from cryptography.hazmat.primitives.asymmetric import padding
        return padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None)

    @staticmethod
    def padding_PSS():
        from cryptography.hazmat.primitives import hashes
        from cryptography.hazmat.primitives.asymmetric import padding
        return padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH)
    
    def sign(self, message: str) -> bytes:
        ''' uses private key to sign a document '''
        from cryptography.hazmat.primitives import hashes
        signature = self.private.sign(
            Ident.to_bytes(message),
            Ident.padding_PSS(),
            hashes.SHA256())
        return signature
    
    def verify(self, message: str, signature: bytes, public_key=None) -> bool:
        ''' uses public key to verify signature '''
        from cryptography.hazmat.primitives import hashes
        public_key = public_key or self.public
        try:
            public_key.verify(
                Ident.to_bytes(signature),
                Ident.to_bytes(message),
                Ident.padding_PSS(),
                hashes.SHA256())
            return True
        except InvalidSignature:
            return False
    
    def encrypt(self, message: str, public_key=None) -> bytes:
        ''' uses public key to encrypt data so private can decrpyt it '''
        public_key = public_key or self.public
        ciphertext = public_key.encrypt(
            Ident.to_bytes(message),
            Ident.padding_OAEP())
        return ciphertext
    
    def decrypt(self, ciphertext: bytes) -> str:
        ''' uses private key to decrypt data '''
        plaintext = self.private.decrypt(
            Ident.to_bytes(ciphertext), 
            Ident.padding_OAEP())
        return plaintext.decode('utf-8')
    
   

#### testing cryptography

In [3]:
persons = {}
persons['a'] = Ident()
persons['b'] = Ident()
persons['c'] = Ident()

In [4]:
persons['a'].private

In [5]:
type(str.encode('A message I want to sign'))

bytes

In [6]:
type('A message I want to sign')

str

In [7]:
persons['a'].sign(b'A message I want to sign')

b'C\xcc\x11\x9cz\x85\x917\xf4\xb2;\x8d5\xf4\x8f\xdd\xabz\xf3\n+a\xe1l\xd7\x9e}\xbfo}\x9b\xb3]U\xbfMM\xb8"P\x19\xe7\x82_\xd7\xd8\xf5Aw\xea\r\xa63\xed\xe9\x1e{\xf7z\x14\r\xb63P\x0c\x98I;\xff\xe3\x83\x9e\n\x80dl\xaf\x84\xc5\xa3\xba\xeeu\'\x8e\xff\x81+9\n\x857|6\xc2\x02\x91^\t\xf3\x04\xec\xdc^l\xcd\xea\x87\x83\xbd\x10\xa5\xb0\x80fT\xe1\xe4~\x9e\xbf\x01\xb6o!\x8c\xe67i]\x0b\xa3\x7f\x04m\x08F`\xa0\x07vc\xab8L\xbc\xb4\xda4\xf9h\xdd\x882\xd4\x8fr9=\x90\xce\xde%\xa1Z\xd0u\x81Mw}\xda\x9b~\xf1t\x93*\xdb\xc3t\ndw4\xbaK\xb7\xab\x02x\xa8E!\xb3\x0bs\xcf\r/\x01\xbb\xf6\xc0\xc8\x90\x8a\xd4\x8e\x10\xf7\x96\xedv\x9f\xd2&aE\x90\xefX\'\xcb\xc4\xd0"\x9d(eb\xf2\xff\x9b\xe3To\xceR\xaa\x1f\xd9\xda\xe2\n+\xe2\x9d?[\x1e\x92#\xe0\x9e\xbd'

In [8]:
persons['a'].sign(message=str.encode('A message I want to sign'))

b'\xbf\x97\xff\xd2"\xae-\x8a.\x90\x91\x11\x19\xe0\x97\x10\xc9p\xc3\xb7G\x0e\n\xa1\xb6\x8dcZDHPe\xc1\xf4\x0e\xe2GE\xe0SH(Q\x06\xff|$\x84\xea\xaa\xab/\xba(u\xac\xbe\xc2p\x8c\xd5~\x91\x11@\xa0qn\x8d\xc6\xeb]=P\x97P^LX\x94\x93\x1d\x9e+\xbe\x19@\xf0\xa51\xf0\xe6km\x1a\x1e\x97\x83\xe6\x8f\xd1\xbcP\xfc7Mc\xf4bw\x15\x98+*\x03\xe0\xce&\x1c\xd0/Ku\xcd\xd1/\xab[b\xcd}\xfa\xce\x17Q\xb9d\xb9\xf4f\xc0\xc2j\xf4\x99\xa3\x0f\x99z\xc13YR|\x01"\xe1\x9b\x02\xeb8?\xeb\xb4D\xea.\x1e?\xbd~E|z**@N\xbf\x8a\x8f\x11\x97\x02\x9e\x95\xd4*\xc5\xd3\xcbO\xbe(\xf5y0ivC\xa4@\xae\xdc\xc1#6\x0c\xd6N\xf2\xd5\xdc\xc0\x8c\xff\xc2eT|\xf3\x8dO\xbd\xa3"\xd3]\xaa]I\x02\x12\x82\x00\xea\xb2\xcf\x02E\x99\xc3\xdb\x94~\xb8\xcf\xd7\xb2\xc3f\xb4\xd7WJ\xba'

In [9]:
persons['a'].encrypt(message='encrypted data')

b'\x92\xb74<\x11\xc0w\x87@\n~4\x13\xe7h\xc00I\xec\xb3\xe2\xca\xd4\xb58\xe8@"\n\xa1CN_\x85\xd8?\xf3Vj\xe9zg\x9c\xcd\x0e\xeaKr[\x05\xe9\xc8\xa6YD|\xc7\xadR\x84\xe7]\\%[\x8fL\xb7\xa0e\x92V\x85?)&\xa7\xdfS\xd8.\xdb\x93\xb7\xa1\x92\xfd\x9e\x97\xf8\xd3\xb7\n\x0f\\\x123\xdd\x80v\x10\xe6Co(\xa7\xec\x9e\x9d=*d\xfd-\xf9\xb2\xc4P\xd2\xd1\xa5\xf8\xbf\xcf\x19\xe4\x890\xedj~|\xb1\xa1\xa8I\'\xf3\xf9[%\xda-\xe3\xeeP\xdb\xc5\xc2 }\xf9\x1d5.\xbf\xb2\x83\xc6\xbd\x06f\xda\x01l,\xfb\xa6\xba+8\xdd\x82^W\xd6\xcay\x9d\xd8(\xffc\xf6\xe3j-\xf8\xa2I\xc6\xef\xddd\xbd\xd6Pk*n0\xd20H*\xb4=\x0c\x10\x96\xbb\x92ML-8>\x88\xd3\x96u\xd9\xd0\xb6-\x04\xd6\x8e\x05\x1b\x9d2\xd4G\x19\x1aM\n\xee\xa9_\x9dQS\x1f\x88E/;e\xa3$O\xaa\xc9\xa3'

In [11]:
persons['a'].verify(
    message='A message I want to sign', 
    signature=persons['a'].sign('A message I want to sign'))

True

In [12]:
persons['a'].decrypt(ciphertext=persons['a'].encrypt(message='encrypted data'))

'encrypted data'

## Permissioned Consensus

It is known how to acheive consensus amongst a group of permissioned entities. Distributed consensus - on a network that is open to anyone - is more difficult to produce. Assuming the network is permissioned, and therefore has a centralized decision maker on who is amitted into the network is the first step in this process. Distributed consensus can be later bootstrapped from this initial assumption.

In [13]:
# object to serve as a prototype node, with all the abilities of a node
class Ident():
    def __init__(self):
        self.private, self.public = Ident.gen_ident()
        everyone = []
    
    ### Encryption #########################################################################################
    
    @staticmethod
    def gen_ident():
        from cryptography.hazmat.backends import default_backend
        from cryptography.hazmat.primitives.asymmetric import rsa
        private_key = rsa.generate_private_key(
            public_exponent=65537,
            key_size=2048,
            backend=default_backend()
        )
        public_key = private_key.public_key()
        return private_key, public_key
    
    @staticmethod
    def to_bytes(string: str):
        if isinstance(string, str):
            return str.encode(string)
        if isinstance(string, bytes):
            return string
        raise TypeError('string to be converted to bytes is not of str or bytes type')
        
    @staticmethod
    def padding_OAEP():
        from cryptography.hazmat.primitives import hashes
        from cryptography.hazmat.primitives.asymmetric import padding
        return padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None)

    @staticmethod
    def padding_PSS():
        from cryptography.hazmat.primitives import hashes
        from cryptography.hazmat.primitives.asymmetric import padding
        return padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH)
    
    def sign(self, message: str) -> bytes:
        ''' uses private key to sign a document '''
        from cryptography.hazmat.primitives import hashes
        signature = self.private.sign(
            Ident.to_bytes(message),
            Ident.padding_PSS(),
            hashes.SHA256())
        return signature
    
    def verify(self, message: str, signature: bytes, public_key=None) -> bool:
        ''' uses public key to verify signature '''
        from cryptography.hazmat.primitives import hashes
        public_key = public_key or self.public
        try:
            public_key.verify(
                Ident.to_bytes(signature),
                Ident.to_bytes(message),
                Ident.padding_PSS(),
                hashes.SHA256())
            return True
        except InvalidSignature:
            return False
    
    def encrypt(self, message: str, public_key=None) -> bytes:
        ''' uses public key to encrypt data so private can decrpyt it '''
        public_key = public_key or self.public
        ciphertext = public_key.encrypt(
            Ident.to_bytes(message),
            Ident.padding_OAEP())
        return ciphertext
    
    def decrypt(self, ciphertext: bytes) -> str:
        ''' uses private key to decrypt data '''
        plaintext = self.private.decrypt(
            Ident.to_bytes(ciphertext), 
            Ident.padding_OAEP())
        return plaintext.decode('utf-8')
    
    ### Permissioned Consensus ######################################################################################
    
    # NEXT
    
    ### prototype helpers ######################################################################################
    
    def meet(self, someone):
        everyone.append(someone)
        everyone = list(set(everyone))
    
    ### protocol logic ######################################################################################
    
    @staticmethod
    def validate_signature(sig):
        pass
        
    @staticmethod
    def validate_transaction(transaction):
        pass
    
    def make_transaction(self):
        pass
    
    def calculate_balance(self, identity):
        return 0
    
    def calculate_reputation(self, idenitity):
        return 0

#### testing permissioned consensus 